In [2]:

# @title
!git clone https://github.com/NickS785/CTAFlow -b v2

!pip install -e CTAFlow
!git clone https://github.com/NickS785/SierraPy
!pip install -e SierraPy


Cloning into 'CTAFlow'...
remote: Enumerating objects: 2983, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 2983 (delta 163), reused 149 (delta 104), pack-reused 2694 (from 2)
Receiving objects: 100% (2983/2983), 17.76 MiB | 6.51 MiB/s, done.
Resolving deltas: 100% (2105/2105), done.
Obtaining file:///content/CTAFlow
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for CTAFlow (pyproject.toml) ... done
  Created wheel for CTAFlow: filename=ctaflow-1.0.0-0.editable-py3-none-any.whl size=7150 sha256=e8bb432f9c7ae505a636e679d15575da912146ad85e6d5f4177dcf23b54eb0a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-wb6b1a9r/wheels/88/a3/d1/a7ec03ac18ed6e567435cf8ef4bb473f62e14fd8ef8f1c5ba1
Successfully built CTAFlow
Cloning into 'SierraPy'...
r

In [3]:
import sys
sys.path.insert(0,"//content/SierraPy")
sys.path.insert(1, "//content/CTAFlow/CTAFlow")
from SierraPy import sierrapy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Opening Returns Prediction with IntradayMomentumThis notebook demonstrates:1. **Regression Models** - LightGBM (baseline & tuned), XGBoost, Random Forest2. **Classification Models** - Binary and multiclass LightGBM classifiers3. **Model Comparison** - Compare performance across different algorithms**Features Used:**- Short-term daily momentum (1d, 5d, 10d, 20d) - properly lagged- HAR-style realized volatility (1d, 5d, 22d)- Opening range volatility (first 60 minutes)- Previous high/low features**Target**: Returns during the opening period (first 60 minutes of session)**Important**: Uses `model.target_data` to ensure consistency and avoid lookahead bias

## Setup and Imports

In [5]:
from __future__ import annotations
import sys
from pathlib import Path
from datetime import time, timedelta
from typing import Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Add parent directories to path
project_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(project_root))
%cd SierraPy
from CTAFlow.CTAFlow.models.intraday_momentum import IntradayMomentum
from CTAFlow.CTAFlow.models.base_models import CTALight
from CTAFlow.CTAFlow.config import INTRADAY_DATA_PATH

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"✓ Imports successful")
print(f"✓ Data path: {INTRADAY_DATA_PATH}")

print(f"✓ Imports successful")
print(f"✓ Data path: {INTRADAY_DATA_PATH}")

/content/SierraPy
Please ensure all dependencies are installed: pip install -r requirements.txt
Please ensure all dependencies are installed: pip install -r requirements.txt
✓ Imports successful
✓ Data path: F:\Data\intraday
✓ Imports successful
✓ Data path: F:\Data\intraday


## Helper Functions

In [6]:
def calculate_opening_returns(
    intraday_df: pd.DataFrame,
    session_open: time = time(8, 30),
    opening_window: timedelta = timedelta(minutes=60),
    price_col: str = "Close",
) -> pd.Series:
    """
    DEPRECATED: Use model.target_data instead.

    This function is kept for reference only. The IntradayMomentum class
    pre-calculates target_data during initialization with proper lagging to
    avoid lookahead bias. Always use model.target_data for consistency.
    """
    if not isinstance(intraday_df.index, pd.DatetimeIndex):
        intraday_df.index = pd.to_datetime(intraday_df.index)

    work_df = pd.DataFrame({'price': intraday_df[price_col]})
    work_df['date'] = work_df.index.normalize()

    session_open_offset = pd.Timedelta(hours=session_open.hour, minutes=session_open.minute)
    work_df['session_start'] = work_df['date'] + session_open_offset
    work_df['session_end'] = work_df['session_start'] + opening_window

    opening_mask = work_df.index >= work_df['session_start']
    opening_data = work_df[opening_mask].groupby('date')['price'].first()

    closing_mask = (work_df.index >= work_df['session_start']) & (work_df.index < work_df['session_end'])
    closing_data = work_df[closing_mask].groupby('date')['price'].last()

    opening_returns = np.log(closing_data / opening_data)
    return opening_returns


def prepare_daily_data(intraday_df: pd.DataFrame) -> pd.DataFrame:
    """Create daily OHLC data from intraday bars."""
    if not isinstance(intraday_df.index, pd.DatetimeIndex):
        intraday_df.index = pd.to_datetime(intraday_df.index)

    daily = intraday_df.resample('1D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).dropna()

    return daily

print("✓ Helper functions defined")

✓ Helper functions defined


## Load Data

In [ ]:
ticker = "RB"  # ticker for file lookup

supplementary_tickers = {"PL": ["PA", "GC"],
                         "RB": ["HO", "CL"],
                         "CS": ["RB", "CL", "HO"]}
from CTAFlow.CTAFlow.data import read_exported_df, read_synthetic_csv
from pathlib import Path

# Load intraday data from CSV
path = Path('/content', 'drive', 'MyDrive', 'intraday', 'CSV')
csv_path = path / f"{ticker}_5min.csv"

supplementary = {p:read_exported_df(path / f"{p}_5min.csv") for p in supplementary_tickers[ticker]}
print(f"Loading {ticker} from {csv_path}")

intraday_data = read_exported_df(csv_path)

print(f"\n✓ Loaded {len(intraday_data):,} bars")
print(f"✓ Date range: {intraday_data.index[0].date()} to {intraday_data.index[-1].date()}")
print(f"\nData preview:")
intraday_data.head()

## Session Configuration

**Centralized session parameters** - Change these values once to affect all models in the notebook

In [ ]:
# =============================================================================
# SESSION CONFIGURATION - Change these values to affect all models
# =============================================================================
from datetime import time, timedelta

# Session timing parameters
SESSION_START = time(hour=2, minute=30)
SESSION_END = time(hour=11, minute=30)
CLOSING_LENGTH = timedelta(minutes=90)
TIMEZONE = "America/Chicago"
SESSION_TARGET = "close"  # or "open"

# Feature engineering time parameters
KEY_RETURN_TIMES = [time(3, 30), time(9, 30)]
OPENING_VOL_PERIOD = timedelta(minutes=60)
TARGET_PERIOD = timedelta(minutes=480)

# Model configuration
USE_GPU = True

print("="*70)
print("SESSION CONFIGURATION")
print("="*70)
print(f"Session: {SESSION_START} to {SESSION_END}")
print(f"Closing period length: {CLOSING_LENGTH}")
print(f"Target period length: {TARGET_PERIOD}")
print(f"Opening volume period: {OPENING_VOL_PERIOD}")
print(f"Key return times: {KEY_RETURN_TIMES}")
print(f"Timezone: {TIMEZONE}")
print(f"GPU enabled: {USE_GPU}")
print("="*70)

## Initialize Model and Extract Target Data

**Important**: We use the model's pre-calculated `target_data` attribute instead of manually calculating target returns. This ensures:
- Consistency with the feature engineering pipeline
- Proper handling of lookahead bias
- Same target calculation used throughout the class

In [ ]:
# Initialize a base model to get target_data
# This creates the target returns with proper lagging to avoid lookahead bias
base_model = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    base_model=CTALight,
    task='regression',
    supplementary_intraday_data=supplementary,
    use_gpu=USE_GPU
)

# Use the pre-calculated target_data from the model
# This is calculated during __init__ with proper date alignment
target_returns = base_model.target_data

# Prepare daily data for feature engineering
daily_df = prepare_daily_data(intraday_data)

print(f"✓ Daily data: {len(daily_df)} days")
print(f"✓ Target returns: {len(target_returns)} days")
print(f"\nTarget Statistics:")
print(f"  Mean: {target_returns.mean():.6f}")
print(f"  Std:  {target_returns.std():.6f}")
print(f"  Min:  {target_returns.min():.6f}")
print(f"  Max:  {target_returns.max():.6f}")

## Feature Engineering Function

In [ ]:
def build_features(model, daily_df, intraday_data, 
                   key_return_times=KEY_RETURN_TIMES,
                   target_period=TARGET_PERIOD,
                   opening_vol_period=OPENING_VOL_PERIOD):
    """Build comprehensive feature set for the model using centralized configuration."""
    model.training_data = pd.DataFrame(index=daily_df.index)

    # Daily momentum features (lagged)
    momentum_feats = model.add_daily_momentum_features(
        daily_df,
        lookbacks=(1, 10, 20)
    )

    # HAR volatility features
    har_feats = model.har_volatility_features(
        intraday_df=intraday_data,
        horizons=(1, 5)
    )

    # Opening range volatility
    opening_vol = model.opening_range_volatility(
        intraday_df=intraday_data,
        period_length=opening_vol_period,
    )

    # Target time returns - use the last key return time as session end start
    session_end_start = key_return_times[-1]
    
    target_time = model.target_time_returns(
        session_end_start, 
        period_length=target_period, 
        add_as_feature=True
    )
    
    t2_time = model.target_time_returns(
        key_return_times, 
        period_length=timedelta(minutes=60), 
        add_as_feature=True
    )

    # Target volumes
    target_volume = model.target_time_volume(
        session_end_start, 
        period_length=target_period, 
        add_as_feature=True
    )
    
    opening_volume = model.target_time_volume(
        key_return_times, 
        period_length=timedelta(minutes=60), 
        add_as_feature=True
    )
    
    # Correlations
    tickers = [*supplementary.keys()]
    corrs = model.intraday_correlation(
        tickers[-1], 
        target_time=session_end_start,  
        n_bars=8, 
        add_as_feature=True
    )

    print(model.training_data.head())
    return model.training_data

print("✓ Feature engineering function defined")

# Part 1: Regression Models

Predict continuous opening period returns using different model configurations.

## Model 1: Baseline LightGBM Regressor

In [ ]:
print("="*70)
print("MODEL 1: BASELINE LIGHTGBM REGRESSOR")
print("="*70)

# Initialize with default CTALight (regression)
model_reg1 = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    base_model=CTALight,
    task='regression',
    supplementary_intraday_data=supplementary,
    use_gpu=USE_GPU
)

# Build features
X_reg1 = build_features(model_reg1, daily_df, intraday_data)
print(f"\nFeatures built: {X_reg1.shape}")
print(f"Feature columns: {list(X_reg1.columns)}")

In [ ]:
# Align features and target
common_idx = X_reg1.index.intersection(target_returns.index)
X = X_reg1.loc[common_idx]
y = target_returns.loc[common_idx]

# Train/test split (80/20 temporal)
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Validation set for early stopping
val_size = int(len(X_train) * 0.2)
X_val = X_train.iloc[-val_size:]
y_val = y_train.iloc[-val_size:]
X_train_fit = X_train.iloc[:-val_size]
y_train_fit = y_train.iloc[:-val_size]

print(f"\nData split:")
print(f"  Train: {len(X_train_fit)} samples")
print(f"  Val:   {len(X_val)} samples")
print(f"  Test:  {len(X_test)} samples")

# Train model
print(f"\nTraining baseline regressor...")
model_reg1.fit(
    X_train_fit, y_train_fit,
    eval_set=(X_val, y_val),
    early_stopping_rounds=100,
    num_boost_round=500
)

# Evaluate
metrics_reg1 = model_reg1.evaluate(X_test, y_test)
print(f"\nBaseline Regression Metrics:")
print(f"  R²:   {metrics_reg1['r2']:.4f}")
print(f"  RMSE: {metrics_reg1['rmse']:.6f}")
print(f"  MAE:  {metrics_reg1['mae']:.6f}")
print(f"  Dir Acc: {metrics_reg1['directional_accuracy']:.2%}")

## Model 2: Tuned LightGBM Regressor (Grid Search)

In [ ]:
print("="*70)
print("MODEL 2: TUNED LIGHTGBM REGRESSOR (GRID SEARCH)")
print("="*70)

model_reg2 = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    task='regression',
    supplementary_intraday_data=supplementary,
    base_model=CTALight,
    use_gpu=USE_GPU
)

# Build features
X_reg2 = build_features(model_reg2, daily_df, intraday_data)
common_idx2 = X_reg2.index.intersection(target_returns.index)
X2 = X_reg2.loc[common_idx2]
y2 = target_returns.loc[common_idx2]

# Same split
split_idx2 = int(len(X2) * 0.8)
X_train2, X_test2 = X2.iloc[:split_idx2], X2.iloc[split_idx2:]
y_train2, y_test2 = y2.iloc[:split_idx2], y2.iloc[split_idx2:]
val_size2 = int(len(X_train2) * 0.2)
X_val2 = X_train2.iloc[-val_size2:]
y_val2 = y_train2.iloc[-val_size2:]
X_train_fit2 = X_train2.iloc[:-val_size2]
y_train_fit2 = y_train2.iloc[:-val_size2]

# Parameter grid
param_grid = {
    'num_leaves': [31, 63],
    'learning_rate': [0.02, 0.07, 0.1],
    'feature_fraction': [0.7, 0.9]
}

print(f"\nRunning grid search with {np.prod([len(v) for v in param_grid.values()])} combinations...")

grid_results = model_reg2.fit_with_grid_search(
    X_train_fit2, y_train_fit2,
    eval_set=(X_val2, y_val2),
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    early_stopping_rounds=100,
    verbose=True
)

In [ ]:
print(f"Best parameters: {grid_results['best_params']}")
print(f"Best mean CV score: {grid_results['best_mean_score']:.6f}")
print(f"Best individual fold: {grid_results['best_fold_score']:.6f}")
print(f"Worst individual fold: {grid_results['worst_fold_score']:.6f}")
print(f"Std across folds: {grid_results['std_score']:.6f}")

# Evaluate
metrics_reg2 = model_reg2.evaluate(X_test2, y_test2)
print(f"\nTuned Regression Metrics:")
print(f"  R²:   {metrics_reg2['r2']:.4f}")
print(f"  RMSE: {metrics_reg2['rmse']:.6f}")
print(f"  MAE:  {metrics_reg2['mae']:.6f}")
print(f"  Dir Acc: {metrics_reg2['directional_accuracy']:.2%}")

print(f"\nImprovement over baseline:")
print(f"  ΔR²: {metrics_reg2['r2'] - metrics_reg1['r2']:.4f}")

## Model 3: XGBoost Regressor

In [ ]:
print("="*70)
print("MODEL 3: XGBOOST REGRESSOR")
print("="*70)
from CTAFlow.CTAFlow.models.base_models import CTAXGBoost, CTARForest

# Initialize with XGBoost
model_reg3 = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    task='regression',
    supplementary_intraday_data=supplementary,
    base_model=CTAXGBoost,  # XGBoost model
)

# Build features
X_reg3 = build_features(model_reg3, daily_df, intraday_data)
common_idx3 = X_reg3.index.intersection(target_returns.index)
X3 = X_reg3.loc[common_idx3]
y3 = target_returns.loc[common_idx3]

# Same split
split_idx3 = int(len(X3) * 0.8)
X_train3, X_test3 = X3.iloc[:split_idx3], X3.iloc[split_idx3:]
y_train3, y_test3 = y3.iloc[:split_idx3], y3.iloc[split_idx3:]
val_size3 = int(len(X_train3) * 0.2)
X_val3 = X_train3.iloc[-val_size3:]
y_val3 = y_train3.iloc[-val_size3:]
X_train_fit3 = X_train3.iloc[:-val_size3]
y_train_fit3 = y_train3.iloc[:-val_size3]

print(f"Data split:")
print(f"  Train: {len(X_train_fit3)} samples")
print(f"  Val:   {len(X_val3)} samples")
print(f"  Test:  {len(X_test3)} samples")

# Train XGBoost model
print(f"\nTraining XGBoost regressor...")
model_reg3.fit_with_grid_search(
    X_train_fit3, y_train_fit3,
)

# Evaluate
metrics_reg3 = model_reg3.evaluate(X_test3, y_test3)
print(f"\nXGBoost Regression Metrics:")
print(f"  R²:   {metrics_reg3['r2']:.4f}")
print(f"  RMSE: {metrics_reg3['rmse']:.6f}")
print(f"  MAE:  {metrics_reg3['mae']:.6f}")
print(f"  Dir Acc: {metrics_reg3['directional_accuracy']:.2%}")

## Model 4: Random Forest Regressor

In [ ]:
print("="*70)
print("MODEL 4: RANDOM FOREST REGRESSOR")
print("="*70)

# Initialize with Random Forest
model_reg4 = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    task='regression',
    supplementary_intraday_data=supplementary,
    base_model=CTARForest
)

# Build features
X_reg4 = build_features(model_reg4, daily_df, intraday_data)
common_idx4 = X_reg4.index.intersection(target_returns.index)
X4 = X_reg4.loc[common_idx4]
y4 = target_returns.loc[common_idx4]

# Same split
split_idx4 = int(len(X4) * 0.8)
X_train4, X_test4 = X4.iloc[:split_idx4], X4.iloc[split_idx4:]
y_train4, y_test4 = y4.iloc[:split_idx4], y4.iloc[split_idx4:]

print(f"Data split:")
print(f"  Train: {len(X_train4)} samples")
print(f"  Test:  {len(X_test4)} samples")

# Train Random Forest model (no early stopping for RF)
print(f"\nTraining Random Forest regressor...")
model_reg4.fit_with_grid_search(X_train4, y_train4, cv=3)

# Evaluate
metrics_reg4 = model_reg4.evaluate(X_test4, y_test4)
print(f"\nRandom Forest Regression Metrics:")
print(f"  R²:   {metrics_reg4['r2']:.4f}")
print(f"  RMSE: {metrics_reg4['rmse']:.6f}")
print(f"  MAE:  {metrics_reg4['mae']:.6f}")
print(f"  Dir Acc: {metrics_reg4['directional_accuracy']:.2%}")

## Regression Comparison Visualization

In [ ]:

# Generate predictions from models
y_pred_reg1 = model_reg1.predict(X_test)
y_pred_reg2 = model_reg2.predict(X_test2)
y_pred_reg3 = model_reg3.predict(X_test3)
y_pred_reg4 = model_reg4.predict(X_test4)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Baseline scatter
axes[0, 0].scatter(y_test, y_pred_reg1, alpha=0.6, s=30, label='Predictions', edgecolors='black', linewidth=0.5)
# Use data range for perfect line instead of min/max which can be outliers
q_low, q_high = y_test.quantile([0.05, 0.95])
axes[0, 0].set_xlabel('Actual Returns', fontsize=11)
axes[0, 0].set_ylabel('Predicted Returns', fontsize=11)
axes[0, 0].set_title(f'Model 1: LightGBM Baseline (R²={metrics_reg1["r2"]:.4f})', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Tuned scatter
axes[0, 1].scatter(y_test2, y_pred_reg2, alpha=0.6, s=30, color='green', label='Predictions', edgecolors='black', linewidth=0.5)
q_low2, q_high2 = y_test2.quantile([0.05, 0.95])
axes[0, 1].set_xlabel('Actual Returns', fontsize=11)
axes[0, 1].set_ylabel('Predicted Returns', fontsize=11)
axes[0, 1].set_title(f'Model 2: LightGBM Tuned (R²={metrics_reg2["r2"]:.4f})', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Model comparison bar chart
model_names = ['LightGBM\nBaseline', 'LightGBM\nTuned', 'XGBoost', 'Random\nForest']
r2_scores = [metrics_reg1['r2'], metrics_reg2['r2'], metrics_reg3['r2'], metrics_reg4['r2']]
colors = ['steelblue', 'green', 'orange', 'purple']
bars = axes[1, 0].bar(model_names, r2_scores, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('R² Score', fontsize=11)
axes[1, 0].set_title('Model Performance Comparison (R²)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].set_ylim([min(r2_scores) - 0.05, max(r2_scores) + 0.05])
# Add value labels on bars
for bar, score in zip(bars, r2_scores):
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{score:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Feature importance (tuned model)
top_features = model_reg2.model.get_feature_importance(importance_type='gain', top_n=20)
features = list(top_features.keys())
importances = list(top_features.values)
# Sort by importance
sorted_idx = sorted(range(len(importances)), key=lambda k: importances[k])
features_sorted = [features[i] for i in sorted_idx]
importances_sorted = [importances[i] for i in sorted_idx]
axes[1, 1].barh(features_sorted, importances_sorted, color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.8)
axes[1, 1].set_xlabel('Importance (Gain)', fontsize=11)
axes[1, 1].set_title('Top 10 Features - LightGBM Tuned Model', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


# Part 2: Classification Models

Predict opening period direction (binary) or regime (multiclass).

## Model 3: Binary Classifier (Up/Down)

In [ ]:
print("="*70)
print("MODEL 3: BINARY CLASSIFIER (UP/DOWN)")
print("="*70)

# Initialize with binary classification task
model_clf_binary = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    supplementary_intraday_data=supplementary,
    base_model=CTALight,
    task='binary_classification'
)

# Build features
X_clf_bin = build_features(model_clf_binary, daily_df, intraday_data)

# Create binary target (0=down, 1=up)
y_clf_binary = model_clf_binary.create_clf_target(
    n_classes=2,
    add_as_feature=False
)

print(f"\nBinary target distribution:")
print(y_clf_binary.value_counts(normalize=True))
print(f"\nFeatures: {X_clf_bin.shape}")

In [ ]:
# Align and split
common_idx_bin = X_clf_bin.index.intersection(y_clf_binary.index)
X_bin = X_clf_bin.loc[common_idx_bin]
y_bin = y_clf_binary.loc[common_idx_bin]

split_idx_bin = int(len(X_bin) * 0.8)
X_train_bin, X_test_bin = X_bin.iloc[:split_idx_bin], X_bin.iloc[split_idx_bin:]
y_train_bin, y_test_bin = y_bin.iloc[:split_idx_bin], y_bin.iloc[split_idx_bin:]

val_size_bin = int(len(X_train_bin) * 0.2)
X_val_bin = X_train_bin.iloc[-val_size_bin:]
y_val_bin = y_train_bin.iloc[-val_size_bin:]
X_train_fit_bin = X_train_bin.iloc[:-val_size_bin]
y_train_fit_bin = y_train_bin.iloc[:-val_size_bin]

print(f"Data split:")
print(f"  Train: {len(X_train_fit_bin)} samples")
print(f"  Val:   {len(X_val_bin)} samples")
print(f"  Test:  {len(X_test_bin)} samples")

# Train binary classifier
print(f"\nTraining binary classifier...")
model_clf_binary.fit(
    X_train_fit_bin, y_train_fit_bin,
    eval_set=(X_val_bin, y_val_bin),
    early_stopping_rounds=50,
    num_boost_round=1000
)

# Predict and convert probabilities to class labels
# LightGBM binary classification returns probabilities for class 1
y_pred_proba_bin = model_clf_binary.predict(X_test_bin)
y_pred_bin = (y_pred_proba_bin > 0.5).astype(int)  # Convert to class labels

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(f"\nBinary Classification Metrics:")
print(f"  Accuracy:  {accuracy_score(y_test_bin, y_pred_bin):.4f}")
print(f"  Precision: {precision_score(y_test_bin, y_pred_bin):.4f}")
print(f"  Recall:    {recall_score(y_test_bin, y_pred_bin):.4f}")
print(f"  F1 Score:  {f1_score(y_test_bin, y_pred_bin):.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test_bin, y_pred_bin, target_names=['Down', 'Up']))

In [ ]:
# Confusion matrix
cm_bin = confusion_matrix(y_test_bin, y_pred_bin)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_bin, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'], ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Binary Classification Confusion Matrix')
plt.tight_layout()
plt.show()

## Model 4: Multiclass Classifier (Down/Neutral/Up)

In [ ]:
print("="*70)
print("MODEL 4: MULTICLASS CLASSIFIER (DOWN/NEUTRAL/UP)")
print("="*70)

# Initialize with multiclass task
model_clf_multi = IntradayMomentum(
    intraday_data=intraday_data,
    session_open=SESSION_START,
    session_end=SESSION_END,
    closing_length=CLOSING_LENGTH,
    tz=TIMEZONE,
    supplementary_intraday_data=supplementary,
    base_model=CTALight,
    task='multiclass',  # Multiclass classification
    num_class=3
)

# Build features
X_clf_multi = build_features(model_clf_multi, daily_df, intraday_data)

# Create multiclass target with thresholds (0=down, 1=neutral, 2=up)
# Use 25th and 75th percentiles as thresholds
lower_threshold = target_returns.quantile(0.4)
upper_threshold = target_returns.quantile(0.6)

y_clf_multi = model_clf_multi.create_clf_target(
    n_classes=3,
    lower_bound=lower_threshold,
    upper_bound=upper_threshold,
    add_as_feature=False
)

print(f"\nThresholds:")
print(f"  Lower (25th %ile): {lower_threshold:.6f}")
print(f"  Upper (75th %ile): {upper_threshold:.6f}")

print(f"\nMulticlass target distribution:")
print(y_clf_multi.value_counts(normalize=True).sort_index())
print(f"\nFeatures: {X_clf_multi.shape}")

In [ ]:
# Align and split
common_idx_multi = X_clf_multi.index.intersection(y_clf_multi.index)
X_multi = X_clf_multi.loc[common_idx_multi]
y_multi = y_clf_multi.loc[common_idx_multi]

split_idx_multi = int(len(X_multi) * 0.8)
X_train_multi, X_test_multi = X_multi.iloc[:split_idx_multi], X_multi.iloc[split_idx_multi:]
y_train_multi, y_test_multi = y_multi.iloc[:split_idx_multi], y_multi.iloc[split_idx_multi:]

val_size_multi = int(len(X_train_multi) * 0.2)
X_val_multi = X_train_multi.iloc[-val_size_multi:]
y_val_multi = y_train_multi.iloc[-val_size_multi:]
X_train_fit_multi = X_train_multi.iloc[:-val_size_multi]
y_train_fit_multi = y_train_multi.iloc[:-val_size_multi]

print(f"Data split:")
print(f"  Train: {len(X_train_fit_multi)} samples")
print(f"  Val:   {len(X_val_multi)} samples")
print(f"  Test:  {len(X_test_multi)} samples")

# Train multiclass classifier
print(f"\nTraining multiclass classifier...")
model_clf_multi.fit(
    X_train_fit_multi, y_train_fit_multi,
    eval_set=(X_val_multi, y_val_multi),
    early_stopping_rounds=50,
    num_boost_round=1000
)

# Predict and convert probabilities to class labels
# LightGBM multiclass returns probabilities for each class
y_pred_proba_multi = model_clf_multi.predict(X_test_multi)
# For multiclass, predict returns (n_samples, n_classes) probabilities
# Use argmax to get the class with highest probability
if y_pred_proba_multi.ndim > 1:
    y_pred_multi = np.argmax(y_pred_proba_multi, axis=1)
else:
    # If it returns 1D, it's already class labels
    y_pred_multi = y_pred_proba_multi.astype(int)

print(f"\nMulticlass Classification Metrics:")
print(f"  Accuracy: {accuracy_score(y_test_multi, y_pred_multi):.4f}")
print(f"  Macro F1: {f1_score(y_test_multi, y_pred_multi, average='macro'):.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test_multi, y_pred_multi,
                          target_names=['Down', 'Neutral', 'Up']))

In [ ]:
# Confusion matrix
cm_multi = confusion_matrix(y_test_multi, y_pred_multi)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm_multi, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Down', 'Neutral', 'Up'],
            yticklabels=['Down', 'Neutral', 'Up'], ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Multiclass Classification Confusion Matrix')
plt.tight_layout()
plt.show()

## Classification Feature Importance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Binary classifier features
top_bin = model_clf_binary.model.get_feature_importance(importance_type='gain', top_n=20)
features_bin = list(top_bin.keys())
importances_bin = list(top_bin.values)
# Sort by importance
sorted_idx_bin = sorted(range(len(importances_bin)), key=lambda k: importances_bin[k])
features_bin_sorted = [features_bin[i] for i in sorted_idx_bin]
importances_bin_sorted = [importances_bin[i] for i in sorted_idx_bin]
axes[0].barh(features_bin_sorted, importances_bin_sorted, color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.8)
axes[0].set_xlabel('Importance (Gain)', fontsize=11)
axes[0].set_title('Top 10 Features - Binary Classifier', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Multiclass classifier features
top_multi = model_clf_multi.model.get_feature_importance(importance_type='gain', top_n=25)
features_multi = list(top_multi.keys())
importances_multi = list(top_multi.values)
# Sort by importance
sorted_idx_multi = sorted(range(len(importances_multi)), key=lambda k: importances_multi[k])
features_multi_sorted = [features_multi[i] for i in sorted_idx_multi]
importances_multi_sorted = [importances_multi[i] for i in sorted_idx_multi]
axes[1].barh(features_multi_sorted, importances_multi_sorted, color='green', alpha=0.7, edgecolor='black', linewidth=0.8)
axes[1].set_xlabel('Importance (Gain)', fontsize=11)
axes[1].set_title('Top 10 Features - Multiclass Classifier', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


# Summary: Model Comparison

Comparison of all models across regression and classification tasks.

In [ ]:
print("="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)

print("\nREGRESSION MODELS:")
print("-" * 70)
print(f"Model 1 (LightGBM Baseline): R² = {metrics_reg1['r2']:.4f}, RMSE = {metrics_reg1['rmse']:.6f}")
print(f"Model 2 (LightGBM Tuned):    R² = {metrics_reg2['r2']:.4f}, RMSE = {metrics_reg2['rmse']:.6f}")
print(f"Model 3 (XGBoost):           R² = {metrics_reg3['r2']:.4f}, RMSE = {metrics_reg3['rmse']:.6f}")
print(f"Model 4 (Random Forest):     R² = {metrics_reg4['r2']:.4f}, RMSE = {metrics_reg4['rmse']:.6f}")

print("\nCLASSIFICATION MODELS:")
print("-" * 70)
print(f"Model 5 (Binary):            Accuracy = {accuracy_score(y_test_bin, y_pred_bin):.4f}, F1 = {f1_score(y_test_bin, y_pred_bin):.4f}")
print(f"Model 6 (Multiclass):        Accuracy = {accuracy_score(y_test_multi, y_pred_multi):.4f}, F1 = {f1_score(y_test_multi, y_pred_multi, average='macro'):.4f}")

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)
print("✓ Target data uses model.target_data to ensure consistency and avoid lookahead bias")
print("✓ IntradayMomentum supports multiple base models (LightGBM, XGBoost, RandomForest)")
print("✓ Feature engineering is model-agnostic (same features for all models)")
print("✓ Grid search can improve LightGBM regression performance")
print("✓ Different models show varying prediction accuracy - compare R² and RMSE")
print("✓ Binary and multiclass classification achieve reasonable directional accuracy")
print("✓ All models properly handle temporal splits to avoid lookahead bias")